In [11]:
################################################################################
#
# RevBayes script: A simple FBD analysis
# 
# This file: Runs the full MCMC ...
#
# authors: Tracy A. Heath, Josh Justison, Joëlle Barido-Sottani, and Walker Pett
#
################################################################################

#######################
# Reading in the Data #
#######################
# Read the full list of taxa (including all fossils and extant species #
taxa <- readTaxonData("../fbd_simple/bears_taxa.tsv")

# Import the morphological character matrix #
# this file contains only the taxa for which morphological characters are available #
morpho <- readDiscreteCharacterData("../fbd_simple/bears_morphology.nex")

## helpers
n_taxa <- taxa.size()
moves = VectorMoves()

##########################################################################################
# Joint Fossilized Birth-Death Process prior on the topology and fossil occurrence times #
##########################################################################################

# Define exponential priors on the birth rate and death rate #
speciation_rate ~ dnExponential(10)
extinction_rate ~ dnExponential(10)

# Specify a scale move on the speciation_rate parameter #
# This move will be applied with 3 different tuning values (lambda) to help improve mixing # 
moves.append( mvScale(speciation_rate, weight=1) )

# Specify a scale move on the extinction_rate parameter #
# This move will be applied with 3 different tuning values (lambda) to help improve mixing # 
moves.append( mvScale(extinction_rate, weight=1) )

# Create deterministic nodes for the diversification and turnover rates so that they can be monitored #
diversification := speciation_rate - extinction_rate
turnover := extinction_rate/speciation_rate


# Fix the probability of sampling parameter (rho) to 1, #
# because all extant bears are represented in this analysis #
rho <- 1.0

# Assume an exponential prior on the rate of sampling fossils (psi) #
psi ~ dnExponential(10) 

# Specify a scale move on the psi parameter #
# This move will be applied with 3 different tuning values (lambda) to help improve mixing # 
moves.append( mvScale(psi, weight=1) )


# The FBD is conditioned on a starting time for the process, which is the origin time #
# Specify a uniform prior on the origin #
origin_time ~ dnUnif(37.0, 55.0)

# Specify a sliding-window move on the origin_time parameter #
# This move will be applied with 3 different window widths (delta) to help improve mixing # 
moves.append( mvSlide(origin_time, weight=1.0) )


### Define the tree-prior distribution as the fossilized birth-death process ###
fbd_tree ~ dnFBDP(origin=origin_time, lambda=speciation_rate, mu=extinction_rate, psi=psi, rho=rho, taxa=taxa)

# Specify moves on the tree and node times #
# These moves update the tree topology 
moves.append( mvFNPR(fbd_tree, weight=15.0) )
moves.append( mvCollapseExpandFossilBranch(fbd_tree, origin_time, weight=6.0) )

# These moves update the node ages #
# Because we are conditioning on the origin time, we must also sample the root node age #
moves.append( mvNodeTimeSlideUniform(fbd_tree, weight=40.0) )
moves.append( mvRootTimeSlideUniform(fbd_tree, origin_time, weight=5.0) )


### Use stratigraphic range data to explicitly sample the fossil occurence times ###
# Use a for loop to create a uniform distribution on the occurence time for each fossil #
# The boundaries of the uniform distribution are specified in the tsv file #
fossils = fbd_tree.getFossils()

   Successfully read one character matrix from file '../fbd_simple/bears_morphology.nex'


In [16]:
extant_taxa <- morpho.taxa()

extant_taxa
fossils

   [ Agriarctos_spp, Ailurarctos_lufengensis, Ailuropoda_melanoleuca, Ballusia_elmensis, Helarctos_malayanus, Indarctos_arctoides, Indarctos_punjabiensis,
   Indarctos_vireti, Kretzoiarctos_beatrix, Melursus_ursinus, Tremarctos_ornatus, Ursavus_brevirhinus, Ursavus_primaevus, Ursus_americanus, Ursus_arctos,
   Ursus_maritimus, Ursus_thibetanus, Zaragocyon_daamsi ]
   [ Ailurarctos_lufengensis, Agriarctos_spp, Indarctos_vireti, Indarctos_punjabiensis, Indarctos_arctoides, Kretzoiarctos_beatrix, Ursavus_primaevus,
   Ursavus_brevirhinus, Zaragocyon_daamsi, Ballusia_elmensis ]


In [1]:
for(i in 1:fossils.size())
{
    t[i] := tmrca(fbd_tree, clade(fossils[i]))

    a_i = fossils[i].getMinAge()
    b_i = fossils[i].getMaxAge()

    F[i] ~ dnUniform(t[i] - b_i, t[i] - a_i)
    F[i].clamp( 0 )
}

# Add a move to sample the fossil times #
moves.append( mvFossilTimeSlideUniform(fbd_tree, origin_time, weight=5.0) )


### Create deterministic nodes to monitor various tree statistics ###
# Monitor the number of sampled ancestors in the FBD-tree #
num_samp_anc := fbd_tree.numSampledAncestors();

                    
# Monitor the age of the extant bears #
clade_extant = clade("Ailuropoda_melanoleuca","Tremarctos_ornatus","Melursus_ursinus",
                    "Ursus_arctos","Ursus_maritimus","Helarctos_malayanus",
                    "Ursus_americanus","Ursus_thibetanus")
age_extant := tmrca(fbd_tree, clade_extant)

# Monitor the age of a particular fossil #
age_Kretzoiarctos_beatrix := tmrca(fbd_tree, clade("Kretzoiarctos_beatrix"))


###########################################
# Binary morphological substitution model #
###########################################
#Create the Q matrix. These data are binary, so we initialize the Jukes-Cantor matrix with
# two states
Q_morpho <- fnJC(2)

#Set up Gamma-distributed rate variation.
alpha_morpho ~ dnExponential( 1.0 )
rates_morpho := fnDiscretizeGamma( alpha_morpho, alpha_morpho, 4 )

#Moves on the parameters to the Gamma distribution.
moves.append( mvScale(alpha_morpho, weight=5.0) )

# We assume a strict morphological clock rate, drawn from an exponential prior #
clock_morpho ~ dnExponential(1.0)

moves.append( mvScale(clock_morpho, weight=4.0) )

### Create the substitution model and clamp with our observed Standard data ###
# Here we use the option siteMatrices=true specify that the vector Q #
# represents a site-specific mixture of rate matrices #
# We also condition on observing only variable characters using coding="variable" #
phyMorpho ~ dnPhyloCTMC(tree=fbd_tree, siteRates=rates_morpho, branchRates=clock_morpho, Q=Q_morpho, type="Standard", coding="variable")
phyMorpho.clamp(morpho)


########
# MCMC #
########

# initialize the model object #
mymodel = model(fbd_tree)

monitors = VectorMonitors()

# Create a vector of monitors #
# 1. for the full model #
monitors.append( mnModel(filename="output/bears.log", printgen=10, exclude = ["F"]) )

# 2. the tree #
monitors.append( mnFile(filename="output/bears.trees", printgen=10, fbd_tree) )

# 3. and a few select parameters to be printed to the screen #
monitors.append( mnScreen(printgen=10, age_extant, num_samp_anc, origin_time) )

# Initialize the MCMC object #
mymcmc = mcmc(mymodel, monitors, moves)

# Run the MCMC #
mymcmc.run(generations=10000)

# Quit RevBayes #
q()

   Missing Variable:	Variable os does not exist

   Successfully read one character matrix from file '../fbd_simple/bears_morphology.nex'
   
   Running MCMC simulation
   This simulation runs 1 independent replicate.
   The simulator uses 11 different moves in a random move schedule with 84 moves per iteration
   

Iter        |      Posterior   |     Likelihood   |          Prior   |     age_extant   |   num_samp_anc   |    origin_time   |    elapsed   |        ETA   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
0           |       -718.923   |       -580.902   |       -138.021   |       3.752787   |              0   |       45.48666   |   00:00:00   |   --:--:--   |
10          |       -553.877   |       -450.452   |       -103.425   |       8.344091   |              5   |       46.25945   |   00:00:00   |   --:--:--   |
20          |        -547.88   |       -453.064   

430         |       -525.343   |       -431.454   |       -93.8894   |       24.82672   |              8   |       49.11536   |   00:00:07   |   00:00:01   |
440         |       -530.876   |        -432.23   |       -98.6455   |       22.59155   |              7   |       48.25916   |   00:00:07   |   00:00:00   |
450         |       -529.907   |         -430.7   |       -99.2072   |       24.38927   |              6   |       47.86861   |   00:00:08   |   00:00:00   |
460         |       -525.334   |       -429.774   |       -95.5601   |       29.85104   |              8   |       47.43859   |   00:00:08   |   00:00:00   |
470         |       -531.939   |       -433.126   |       -98.8132   |       27.79715   |              8   |       47.39625   |   00:00:08   |   00:00:00   |
480         |       -528.024   |       -429.536   |       -98.4886   |       23.58886   |              7   |        48.4422   |   00:00:08   |   00:00:00   |
490         |       -523.568   |       -429.934   | 